#  11. Solidity 제어, 데이터구조

* Last Updated 20190605WEB1200 20190120SUN0000 20180707SAT1300 20170112
* todo: 1) ```watch``` 2) library ```myLib+StringUtils``` 3) 계정에서 발생한 거래의 목록? 4) registry

## 9.1 학습내용

### 9.1.1 목표
 
* solidity 배우기

### 9.1.2 목차

* 9.6 [제어](#9.6)
    * 문제 Hello World Contract - 이름정하고, 헬로우 할 때 알아내기
* 9.7 데이터구조
    * 9.7.1 Array
    * 9.7.2 [mapping](#S.5.2)
    * 실습 Members
    * 실습 BankV4
* 9.8 [Exception](#9.6)


## 9.6 제어
<a id='11.4'></a>


### 9.6.1 반복문

반복마다 비용이 소요된다.
무한반복은 사용하면 안된다. gas가 반복마다 소비되고 한도 gasLimit를 초과하게 되면 실행이 중지된다.

* for문
```python
function getSum() public returns(uint) {
    uint sum=0;
    for(uint i=0;i<10;i++)
        sum+=i;
    return sum;
}
```

* while문
```python
function getSum() public returns(uint) {
    uint sum=0;
    uint i=0;
    do {
        sum+=i;
        i++;
    } while(i<10);
    return sum;
}
```

* switch and goto를 제외하고, 다른 언어에서 제공하는 if, else, while, do, for, break, continue, return, ?

```python
if(condition) {
    ...
} else if {
    ...
}
```

Operation name | Gas Cost | Function
-----|-----|-----
step	| 1	| Default amount of gas to pay for an execution cycle.
stop 	| 0	| Nothing paid for the SUICIDE operation.
sha3 	| 20 	| Paid for a SHA3 operation.
sload 	| 20 	| Paid for a SLOAD operation.
sstore 	| 100 	| Paid for a normal SSTORE operation (doubled or waived sometimes).
balance 	| 20 	| Paid for a BALANCE operation
create 	| 100 	| Paid for a CREATE operation
call 	| 20 	| Paid for a CALL operation.
memory 	| 1	| Paid for every additional word when expanding memory
txdata 	| 5 	| Paid for every byte of data or code for a transaction
transaction 	| 500 	| Paid for every transaction

## 실습: Hello World Contract - 이름정하고, 헬로우 할 때 알아내기 

* "Hello world!"를 출력하는 프로그램.
* 로컬에서 command line으로 컴파일하기 - 속성, 생성자, 함수를 갖추어 본다.

###  단계 1: 컨트랙 개발

Line | 설명
-----|-----
1 | major version 5 이상 최신으로 컴파일
2 | contract 명. 파일명과 반드시 일치할 필요가 없다.
3 | 속성 hello. 누구나사용할 수 있도록 public
4 | 속성 address. private으로 자신만 사용할 수 있게 선언.
6 | event로서 문자열 인자를 출력한다. Solidity에는 print문이 별도로 없다. 문자열을 반환하거나 event문으로 출력할 수 있다.
7 | 생성자로서 외부에서 인자를 받아 속성을 초기화. constructor는 public 또는 internal로 선언해야 한다.
8 ~ 9 | 속성 값을 넣어준다. msg는 전역변수로서 msg.sender는 전송자의 계정주소를 말한다. 생성자의 msg.sender를 컨트랙의 소유주owner로 등록한다.
11 ~ 13 | sayHello() 함수는 속성 값을 읽는다. 따라서 view로 선언한다. 인자는 함수 내에서만 잠시 사용하므로 memory로 선언
14 ~ 17 | modifier 함수. msg.sender와 owner가 일치하지 않으면 예외처리한다. 아니면 계속 실행.
20 ~ 30 | printOwner() 함수는 event함수를 가지고 있으므로 view로 선언하지 않는다. view와 event는 같이 사용할 수 없다. event는 상태를 변경하기 때문이다.

> event
> event는 상태를 변경하는 함수이다. 따라서 transaction에서만 사용할 수 있다.
transaction 함수는 마이닝이 되어야 하는 함수이므로 즉시 값을 반환하는 return을 사용하는 것이 의미가 없다.
반면에 call함수는 마이닝없이 즉시 실행되는 함수이므로 event를 발생시키는 것 역시 

* public 속성은 다음과 같이 볼 수 있다.
> h.owner.call();
'0x2e49e21e708b7d83746ec676a4afda47f1a0d693'

In [13]:
%%writefile src/sHello1.sol
pragma solidity ^0.5.0;
contract sHelloV1 {
    string public hello;
    address private owner;
    //event printLog(address _addr); BigNumber error in printing address
    event printLog(string s);
    constructor(string memory _hello) public {
        hello=_hello;
        owner=msg.sender;
    }
    function sayHello() view public returns(string memory) {
        return hello;
    }
    modifier isOwner() {
        if (msg.sender != owner) {
            revert();
        }
        _; //continue executing rest of method body
    }
    function printOwner() isOwner public returns(string memory) {
        string memory s="";
        if (msg.sender==owner) {
            s="msg.sender is the same as the owner";
            emit printLog(s);
        } else {
            s="msg.sender is different from the owner";
            emit printLog(s);
        }
        return s;
    }
}

Overwriting src/sHello1.sol


### 단계 2: 컴파일
* solc를 사용해서 abi, bin을 구한다.
* abi는 ABI Application Binary Interface. ABI는 컴파일 시점에 생성되고, 런타임 동적으로 호출된다.

In [15]:
!solc --abi src/sHello1.sol


======= src/sHello1.sol:sHelloV1 =======
Contract JSON ABI 
[{"constant":true,"inputs":[],"name":"hello","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"printOwner","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"sayHello","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"inputs":[{"name":"_hello","type":"string"}],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"name":"s","type":"string"}],"name":"printLog","type":"event"}]


In [ ]:
!solc --bin src/sHello1.sol

### 단계 3: 배포

* bin앞에 0x를 붙여준다.
* constructor 인자를 new()에 넣어준다.

* gas를 지급해야 하는 등 비용이 발생하는 경우에는 반드시 web3.personal.unlockAccount()를 해주어야 한다.
* browser-solidity를 사용하면, Web3 deploy를 복사해서 사용할 수 있다. abi, bin을 넣은 javascript명령문을 발견할 수 있다.

In [17]:
%%writefile src/sHello1.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    // set the provider you want from Web3.providers
    web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
}
//var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var _abiArray=[{"constant":true,"inputs":[],"name":"hello","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"printOwner","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"sayHello","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"inputs":[{"name":"_hello","type":"string"}],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"name":"s","type":"string"}],"name":"printLog","type":"event"}];
//convert string to json
//var _abiArray=JSON.parse(_abiStr);
var _bin="608060405234801561001057600080fd5b5060405161077f38038061077f8339810180604052602081101561003357600080fd5b81019080805164010000000081111561004b57600080fd5b8281019050602081018481111561006157600080fd5b815185600182028301116401000000008211171561007e57600080fd5b5050929190505050806000908051906020019061009c9291906100e4565b5033600160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555050610189565b828054600181600116156101000203166002900490600052602060002090601f016020900481019282601f1061012557805160ff1916838001178555610153565b82800160010185558215610153579182015b82811115610152578251825591602001919060010190610137565b5b5090506101609190610164565b5090565b61018691905b8082111561018257600081600090555060010161016a565b5090565b90565b6105e7806101986000396000f3fe608060405234801561001057600080fd5b506004361061005e576000357c01000000000000000000000000000000000000000000000000000000009004806319ff1d21146100635780638e69c26c146100e6578063ef5fb05b14610169575b600080fd5b61006b6101ec565b6040518080602001828103825283818151815260200191508051906020019080838360005b838110156100ab578082015181840152602081019050610090565b50505050905090810190601f1680156100d85780820380516001836020036101000a031916815260200191505b509250505060405180910390f35b6100ee61028a565b6040518080602001828103825283818151815260200191508051906020019080838360005b8381101561012e578082015181840152602081019050610113565b50505050905090810190601f16801561015b5780820380516001836020036101000a031916815260200191505b509250505060405180910390f35b6101716104d0565b6040518080602001828103825283818151815260200191508051906020019080838360005b838110156101b1578082015181840152602081019050610196565b50505050905090810190601f1680156101de5780820380516001836020036101000a031916815260200191505b509250505060405180910390f35b60008054600181600116156101000203166002900480601f0160208091040260200160405190810160405280929190818152602001828054600181600116156101000203166002900480156102825780601f1061025757610100808354040283529160200191610282565b820191906000526020600020905b81548152906001019060200180831161026557829003601f168201915b505050505081565b6060600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff161415156102e857600080fd5b606060206040519081016040528060008152509050600160009054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff16141561041057606060405190810160405280602381526020016105736023913990507f68d255efcaa062895d3889d0e53f99389409361c6f0735ef9542be5ca11d1343816040518080602001828103825283818151815260200191508051906020019080838360005b838110156103d15780820151818401526020810190506103b6565b50505050905090810190601f1680156103fe5780820380516001836020036101000a031916815260200191505b509250505060405180910390a16104c9565b606060405190810160405280602681526020016105966026913990507f68d255efcaa062895d3889d0e53f99389409361c6f0735ef9542be5ca11d1343816040518080602001828103825283818151815260200191508051906020019080838360005b8381101561048e578082015181840152602081019050610473565b50505050905090810190601f1680156104bb5780820380516001836020036101000a031916815260200191505b509250505060405180910390a15b8091505090565b606060008054600181600116156101000203166002900480601f0160208091040260200160405190810160405280929190818152602001828054600181600116156101000203166002900480156105685780601f1061053d57610100808354040283529160200191610568565b820191906000526020600020905b81548152906001019060200180831161054b57829003601f168201915b505050505090509056fe6d73672e73656e646572206973207468652073616d6520617320746865206f776e65726d73672e73656e64657220697320646966666572656e742066726f6d20746865206f776e6572a165627a7a723058209316e802cd73c68725b0b850a5f5340b8df488303c883184f8a8ef52d8bdd4850029";
var _contract = web3.eth.contract(_abiArray);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
var _instance=_contract.new("hello world",{data:'0x'+_bin,from:web3.eth.accounts[0],gas:1000000}, function(err, contract) {
    if (!err) {
        console.log("contractAddress: ", contract.address);
        console.log("transactionHash: ", contract.transactionHash);
    }
});


Overwriting src/sHello1.js


In [18]:
!node src/sHello1.js

contractAddress:  undefined
transactionHash:  0x2ef00671e310f594dca636acbc352ce61463e78110e8098b0d483c8374643248
contractAddress:  0x96d8b65b4586b090840d6587c216d4a69071f405
transactionHash:  0x2ef00671e310f594dca636acbc352ce61463e78110e8098b0d483c8374643248


* node로 sHello1.js를 실행하면 화면좌측에 보이듯이 transaction이 생성되고 그 hash값 이 주어진다.

우측 geth 단말에서 보듯이 대기중 transaction hash 값이 서로 일치한다.
마이닝을 하고 나면 contract 주소가 주어진다.

```
INFO [01-08|11:37:07.548] Submitted contract creation
fullhash=0x2ef00671e310f594dca636acbc352ce61463e78110e8098b0d483c8374643248
contract=0x96d8b65B4586B090840d6587C216D4A69071F405
```

![alt text](figures/sol_helloV1Web3DeployMining.png "deploy hello.sol using web3 with mining")

### 단계 4: 사용

In [2]:
%%writefile src/sHello1Use.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var _abiArray=[{"constant":true,"inputs":[],"name":"hello","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"printOwner","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"sayHello","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"inputs":[{"name":"_hello","type":"string"}],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"name":"s","type":"string"}],"name":"printLog","type":"event"}];
//var _abiArray=JSON.parse(_abiArray);
var shelloContract = web3.eth.contract(_abiArray);
var shello = shelloContract.at("0x96d8b65b4586b090840d6587c216d4a69071f405");
var event = shello.printLog(function (error, result) {
    if (!error) {
        console.log("sHello1: event triggered =====> ",result);
        process.exit(1);
    }
});
//console.log(shello.printOwner.call());
console.log(shello.printOwner({from:web3.eth.accounts[0],gas:100000}));

Overwriting src/sHello1Use.js


event의 인자는 transaction log에 저장이 된다.
이벤트는 거래가 마이닝되어 블럭에 포함되어야만 발생된다.
트랙잭션이 pending되어 완료가 되지 않으면, 로그에 포함되지 않기 때문에 여기서 event가 발생하는지 지켜보는 경우  기다리게 된다.
단 이벤트가 발생하는 순서는 지켜진다.

단순하게 블록체인의 transaction이 아닌 ```call()```은 이벤트를 발생하지 않게 된다.
```python
shello.printOwner.call()
```

In [3]:
!node src/sHello1Use.js

0xf2641af8884e1d49e650da2f6cf11e996145b8d656d9bf507c3bf83a72b98527
sHello1: event triggered =====>  { address: '0x96d8b65b4586b090840d6587c216d4a69071f405',
  blockNumber: 88,
  transactionHash: '0xf2641af8884e1d49e650da2f6cf11e996145b8d656d9bf507c3bf83a72b98527',
  transactionIndex: 0,
  blockHash: '0xa525a09ae60c39b1b57ee1077169ef0362d564fc953b122caf904da4b20454d6',
  logIndex: 0,
  removed: false,
  event: 'printLog',
  args: { s: 'msg.sender is the same as the owner' } }


이벤트 로그 event log in REMIX web3 deploy
0.5.0으로 컴파일
* Environment를 Javascript VM으로
* Deploy버튼 옆에 생성자 함수를 호출하기 위해 문자열 "hello jsl"을 넣는다. 문자열에 따옴표를 잊지 않는다.
* Deploy버튼을 누르면 "Deployed Contracts" 아래, sHello 객체가 나타난다. 괄호안 메모리는 실제 블록체인에 배포되지 않았다는 의미 (Javascript VM이므로)
* 속성 및 함수 버튼들을 눌러 결과가 출력되는지 보자.
* printOwner함수 버튼을 누르면 화면 우측 하단을 잘 살펴보자. 특히 굵은 글씨 "logs"를 누르면 하단에 표가 펼쳐지고 이벤트 로그를 볼 수 있다.

![alt text](figures/sol_eventREMIX.png "geth download page")

### 조금 고쳐서 블록체인으로 올려서 사용하기

* string을 합치는 함수는 없다. 인자를 출력하는 함수로 고쳤다.
hello + world와 같이 할 수 없다.
* kill()함수를 만들었다. selfdestruct()를 사용하여 생성자가 컨트랙을 삭제할 수 있다. 컨트랙은 Read Only로서, 만들어지면 영구히 남게된다.

In [29]:
%%writefile src/sHello2.sol
pragma solidity ^0.4.0;
contract sHello2 {
    address owner;
    event printLog(string _s);
    function sHello2() {
        owner=msg.sender;
    }
    function sayHello(string _hello) constant returns(string) {
        printLog(_hello);
        return _hello;
    }
    function kill() { if (msg.sender == owner) selfdestruct(owner); }
}




Overwriting src/sHello2.sol


* 위 컨트랙을 컴파일하였다.
컴파일 바이너리 code는 동일한 contract이지만 공백과 같은 whitespace를 포함하면 달라질 수 있다.
따라서 배포할 경우 컴파일 code와 동일하도록 유의해야 한다.
컴파일은 solc로, 배포할 경우 Browser Solidity로 다르게 하면  
* 컴파일된 컨트랙을 실행하려면, evm이 필요하다.
    * 네트워크를 올리고, 블록체인을 준비한다.
    * 거기에 byte code를 올려서
    * 블록체인의 주소를 통해 사용한다.

In [1]:
!solc src/sHello2.sol --combined-json abi,bin > src/sHello2.json

src/sHello2.sol:5:5: Warning: Defining constructors as functions with the same name as the contract is deprecated. Use "constructor(...) { ... }" instead.
    function sHello2() {
    ^ (Relevant source part starts here and spans across multiple lines).
src/sHello2.sol:9:9: Warning: Invoking events without "emit" prefix is deprecated.
        printLog(_hello);
        ^--------------^
src/sHello2.sol:5:5: Warning: No visibility specified. Defaulting to "public". 
    function sHello2() {
    ^ (Relevant source part starts here and spans across multiple lines).
src/sHello2.sol:8:5: Warning: No visibility specified. Defaulting to "public". 
    function sayHello(string _hello) constant returns(string) {
    ^ (Relevant source part starts here and spans across multiple lines).
src/sHello2.sol:12:5: Warning: No visibility specified. Defaulting to "public". 
    function kill() { if (msg.sender == owner) selfdestruct(owner); }
    ^-----------------------------------------------------------

In [2]:
!cat src/sHello2.json

{"contracts":{"sHello2":{"abi":"[{\"constant\":false,\"inputs\":[],\"name\":\"kill\",\"outputs\":[],\"payable\":false,\"type\":\"function\"},{\"constant\":true,\"inputs\":[{\"name\":\"_hello\",\"type\":\"string\"}],\"name\":\"sayHello\",\"outputs\":[{\"name\":\"\",\"type\":\"string\"}],\"payable\":false,\"type\":\"function\"},{\"inputs\":[],\"payable\":false,\"type\":\"constructor\"},{\"anonymous\":false,\"inputs\":[{\"indexed\":false,\"name\":\"_s\",\"type\":\"string\"}],\"name\":\"printLog\",\"type\":\"event\"}]","bin":"606060405234610000575b33600060006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff1602179055505b5b6102ba8061005c6000396000f30060606040526000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff16806341c0e1b514610049578063c3a9b1c514610058575b610000565b3461000057610056610136565b005b34610000576100ad600480803590602001908201803590602001908080601f016020809104026020016040519081016040528093929190


contractName=Object.keys(_json.contracts) 으로 contract명을 구함 (상대적 경로를 가진)
```python
> contractName=Object.keys(_json.contracts)
[ 'src/sHello2.sol:sHello2' ]
> _json.contracts[contractName].abi
'[{"constant":false,"inputs":[],"name":"kill","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[{"name":"_hello","type":"string"}],"name":"sayHello","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"name":"_s","type":"string"}],"name":"printLog","type":"event"}]'
> JSON.parse(_json.contracts[contractName].abi);
[ { constant: false,
    inputs: [],
    name: 'kill',
    outputs: [],
    payable: false,
    stateMutability: 'nonpayable',
    type: 'function' },
  { constant: true,
    inputs: [ [Object] ],
    name: 'sayHello',
    outputs: [ [Object] ],
    payable: false,
    stateMutability: 'view',
    type: 'function' },
  { inputs: [],
    payable: false,
    stateMutability: 'nonpayable',
    type: 'constructor' },
  { anonymous: false,
    inputs: [ [Object] ],
    name: 'printLog',
    type: 'event' } ]
```

In [2]:
%%writefile src/sHello2.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var fs=require('fs');
var _str = fs.readFileSync("src/sHello2.json");
var _json=JSON.parse(_str);
var _abiArray=JSON.parse(_json.contracts.sHello2.abi);
var _bin=_json.contracts.sHello2.bin;
var _contract = web3.eth.contract(_abiArray);
var _instance=_contract.new({data:'0x'+_bin,from:web3.eth.accounts[0],gas:1000000}, function(err, contract) {
    if (!err && contract.address) {
        console.log("contractAddress: ", contract.address);
        console.log("transactionHash: ", contract.transactionHash);
    }
});




Overwriting src/sHello2.js


In [3]:
!node src/sHello2.js

/home/jsl/Code/git/bb/jsl/bitcoin/src/sHello2.js:6
var _abiArray=JSON.parse(_json.contracts.sHello2.abi);
                                                ^

TypeError: Cannot read property 'abi' of undefined
    at Object.<anonymous> (/home/jsl/Code/git/bb/jsl/bitcoin/src/sHello2.js:6:49)
    at Module._compile (module.js:410:26)
    at Object.Module._extensions..js (module.js:417:10)
    at Module.load (module.js:344:32)
    at Function.Module._load (module.js:301:12)
    at Function.Module.runMain (module.js:442:10)
    at startup (node.js:136:18)
    at node.js:966:3


In [13]:
%%writefile src/sHello2Use.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var fs=require('fs');
var _str = fs.readFileSync("src/sHello2.json");
var _json=JSON.parse(_str);
var _abiArray=JSON.parse(_json.contracts.sHello2.abi);
var _contract = web3.eth.contract(_abiArray);
var sHello2 = _contract.at("0xf22a4ada3e5968646efe489c07d1499530e58dda");
console.log(sHello2.sayHello.call("hello world!"));

Overwriting src/sHello2Use.js


In [14]:
!node src/sHello2Use.js

hello world!


## 9.7 데이터구조

### 9.7.1 array

* 배열은 순서가 있는 데이터로서 인덱스를 사용한다.

* 고정배열

고정배열은 배열이 저장하는 데이터의 개수를 사전에 정할 수 있는 경우에 사용한다.

```python
string[3] cities1=["Seoul", "Sydney", "Tokyo"];
```

괄호 안에 개수를 적어서 초기화할 수 있다.
```python
int[] memory num=new int[](3);
```

* 동적배열

저장하는 데이터의 개수를 미리 정할 수 없고 나중에 정해진다.
```string```은 그 자체로 동적배열이다.
동적배열은 storage함수에만 사용할 수 있고, memory 변수에는 사용할 수 없다.
Storage 배열에는 push() 함수를 사용하여 데이터를 추가할 수 있다.

```python
string[] cities2;
cities2.push("New York");
cities2.push("Beijing");
```

* 다차원배열

정수의 동적 2차원배열이다.
앞의 3은 요소의 개수를, 뒤는 배열의 개수를 의미한다.
다음 예는 3개의 요소를 가진 배열이 2개라는 선언이다.

```python
uint[3][] marks=[[100, 80, 95],[20,30,40]];
```

* 배열의 반환
    * 상태변수는 반환을 할 수 있다.
    * **동적 배열은 그 크기를 알 수 없으므로 반환을 할 수 없다**. 배열을 반환하는 경우 사전에  크기를 알아야 한다.
    * array memory 반환 pragma experimental ABIEncoderV2;

* stat var를 변경하는 경우에는 ```view```라고 정의해서는 안된다.
* 삭제
```python
delete array[index];
```
// If we have data = [11, 12, 13, 14]
delete data[2];
// After deleting we have data = [11, 12, 0, 14]


* ```length```를 사용하여 배열 크기를 알 수 있다.
* ```push```는 ```storage``` 배열에만 사용하고 (즉 상태변수의 배열), ```memory``` 배열에는 사용할 수 없다.

## 실습 배열

* 아래 ```getMathAbove70()```는 REMIX에서는 실행되어 결과가 올바르게 출력되나, 배포한 후 함수를 호출하면 빈 배열만 출력된다.
* ```getCities()```는 동적배열을 반환하는 함수이다 (string의 배열이므로 동적 배열). 동적배열은 반환할 수 없다.

In [1]:
%%writefile src/ArrayTest.sol
pragma solidity ^0.5.0;
pragma experimental ABIEncoderV2;

contract ArrayTest {
    int[5] mathMarks;
    uint[3][] marks=[[100, 80, 95],[20,30,40]];
    string[3] cities1=["Seoul", "Sydney", "Tokyo"];
    string[] cities2=new string[](2);
    string[] cities3;
    function setMathMarks() public {
        mathMarks=[100,60,95,50,80];
    }
    //run setMathMarks() beforehand
    function getMathAbove70() view public returns(int[] memory) {
        //compute lengthOfMathAbove70
        uint8 counter=0;
        uint8 lengthOfMathAbove70=0;
        for(uint8 i=0;i<mathMarks.length;i++)
            if(mathMarks[i]>70) counter++;
        lengthOfMathAbove70=counter;
        //move math marks above 70
        int[] memory mathAbove70=new int[](lengthOfMathAbove70);
        counter=0;
        for(uint i=0;i<mathMarks.length;i++) {
            if(mathMarks[i]>70) {
                mathAbove70[counter]=mathMarks[i];
                counter++;
            }
        }
        return mathAbove70;
    }
    function updateMarks() public returns(uint[3][] memory){
        marks[0][0]=90;
        return marks;
    }
    function setCities() public {
        cities2[0]="New York";
        cities2.push("Busan");
        cities3.push("New York");
        cities3.push("Beijing");
    }
    //dynamic array return needs "pragma experimental ABIEncoderV2;"
    function getCities() view public returns(string[] memory){
        return cities2;
    }
    function getMarksArr() view public returns(uint[3][] memory) {
        return marks;
    }
    function getMarksLength() view public returns(uint) {
        return marks.length;
    }
    function getArrMemory() pure public returns(uint[] memory) {
        uint[] memory num=new uint[](3);
        for (uint i=0; i<num.length; i++)
            num[i]=i;       //push() not allowed for array memeory
        return num;
    }
    //string is a dynamic array itself, which can not be returned.
    //function setArrMemory() view public returns(string[] memory) {
    function getDynamicArrMemory() view public {
        //arr storage in memory not allowed
        //error: string[2] places = ["9000", "Sydney"];
        string[2] memory places = ["9000", "Sydney"];
        //array memory push not allowed
        //places.push("Seoul");
        places[0]="Seoul";
        //return places;
    }
}

Writing src/ArrayTest.sol


In [2]:
!solc --abi --bin src/ArrayTest.sol


======= src/ArrayTest.sol:ArrayTest =======
Binary: 
608060405260405180604001604052806040518060600160405280606460ff168152602001605060ff168152602001605f60ff1681525081526020016040518060600160405280601460ff168152602001601e60ff168152602001602860ff16815250815250600590600262000077929190620001a9565b5060405180606001604052806040518060400160405280600581526020017f53656f756c00000000000000000000000000000000000000000000000000000081525081526020016040518060400160405280600681526020017f5379646e6579000000000000000000000000000000000000000000000000000081525081526020016040518060400160405280600581526020017f546f6b796f0000000000000000000000000000000000000000000000000000008152508152506006906003620001449291906200020e565b5060026040519080825280602002602001820160405280156200017c57816020015b6060815260200190600190039081620001665790505b50600990805190602001906200019492919062000268565b50348015620001a257600080fd5b506200048b565b828054828255906000526020600020906003028101928215620001fb579160200282015b82811115620001fa578251

In [3]:
%%writefile src/ArrayTestDeploy.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
}
var _abiArray=[{"constant":true,"inputs":[],"name":"getMarksArr","outputs":[{"name":"","type":"uint256[3][]"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"getMathAbove70","outputs":[{"name":"","type":"int256[]"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"getArrMemory","outputs":[{"name":"","type":"uint256[]"}],"payable":false,"stateMutability":"pure","type":"function"},{"constant":true,"inputs":[],"name":"getCities","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"setCities","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getMarksLength","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"setMathMarks","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"updateMarks","outputs":[{"name":"","type":"uint256[3][]"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getDynamicArrMemory","outputs":[],"payable":false,"stateMutability":"pure","type":"function"}];
var _bin="608060405260405180604001604052806040518060600160405280606460ff168152602001605060ff168152602001605f60ff1681525081526020016040518060600160405280601460ff168152602001601e60ff168152602001602860ff16815250815250600590600262000077929190620001a9565b5060405180606001604052806040518060400160405280600581526020017f53656f756c00000000000000000000000000000000000000000000000000000081525081526020016040518060400160405280600681526020017f5379646e6579000000000000000000000000000000000000000000000000000081525081526020016040518060400160405280600581526020017f546f6b796f0000000000000000000000000000000000000000000000000000008152508152506006906003620001449291906200020e565b5060026040519080825280602002602001820160405280156200017c57816020015b6060815260200190600190039081620001665790505b50600990805190602001906200019492919062000268565b50348015620001a257600080fd5b506200048b565b828054828255906000526020600020906003028101928215620001fb579160200282015b82811115620001fa57825182906003620001e9929190620002cf565b5091602001919060030190620001cd565b5b5090506200020a919062000319565b5090565b826003810192821562000255579160200282015b8281111562000254578251829080519060200190620002439291906200034a565b509160200191906001019062000222565b5b509050620002649190620003d1565b5090565b828054828255906000526020600020908101928215620002bc579160200282015b82811115620002bb578251829080519060200190620002aa9291906200034a565b509160200191906001019062000289565b5b509050620002cb9190620003d1565b5090565b826003810192821562000306579160200282015b8281111562000305578251829060ff16905591602001919060010190620002e3565b5b50905062000315919062000402565b5090565b6200034791905b808211156200034357600081816200033991906200042a565b5060030162000320565b5090565b90565b828054600181600116156101000203166002900490600052602060002090601f016020900481019282601f106200038d57805160ff1916838001178555620003be565b82800160010185558215620003be579182015b82811115620003bd578251825591602001919060010190620003a0565b5b509050620003cd919062000402565b5090565b620003ff91905b80821115620003fb5760008181620003f191906200043f565b50600101620003d8565b5090565b90565b6200042791905b808211156200042357600081600090555060010162000409565b5090565b90565b50600081556001016000815560010160009055565b50805460018160011615610100020316600290046000825580601f1062000467575062000488565b601f01602090049060005260206000209081019062000487919062000402565b5b50565b610add806200049b6000396000f3fe608060405234801561001057600080fd5b50600436106100935760003560e01c8063a77d717711610066578063a77d717714610259578063a7ad780e14610263578063a8bdea1d14610281578063cec11e681461028b578063d8260e201461030b57610093565b8063479998e7146100985780637a551ab2146101185780638e435ee814610177578063a1317631146101d6575b600080fd5b6100a0610315565b60405180806020018281038252838181518152602001915080516000925b818410156101075782846020026020010151600360200280838360005b838110156100f65780820151818401526020810190506100db565b5050505090500192600101926100be565b925050509250505060405180910390f35b6101206103a5565b6040518080602001828103825283818151815260200191508051906020019060200280838360005b83811015610163578082015181840152602081019050610148565b505050509050019250505060405180910390f35b61017f6104a2565b6040518080602001828103825283818151815260200191508051906020019060200280838360005b838110156101c25780820151818401526020810190506101a7565b505050509050019250505060405180910390f35b6101de610515565b6040518080602001828103825283818151815260200191508051906020019080838360005b8381101561021e578082015181840152602081019050610203565b50505050905090810190601f16801561024b5780820380516001836020036101000a031916815260200191505b509250505060405180910390f35b6102616105cd565b005b61026b610781565b6040518082815260200191505060405180910390f35b61028961078e565b005b6102936107dc565b60405180806020018281038252838181518152602001915080516000925b818410156102fa5782846020026020010151600360200280838360005b838110156102e95780820151818401526020810190506102ce565b5050505090500192600101926102b1565b925050509250505060405180910390f35b61031361089b565b005b60606005805480602002602001604051908101604052809291908181526020016000905b8282101561039c5783829060005260206000209060030201600380602002604051908101604052809291908260038015610388576020028201915b815481526020019060010190808311610374575b505050505081526020019060010190610339565b50505050905090565b60606000809050600080905060008090505b60058160ff1610156103f357604660008260ff16600581106103d557fe5b015413156103e65782806001019350505b80806001019150506103b7565b5081905060608160ff1660405190808252806020026020018201604052801561042b5781602001602082028038833980820191505090505b5090506000925060008090505b60058110156104985760466000826005811061045057fe5b0154131561048b576000816005811061046557fe5b0154828560ff168151811061047657fe5b60200260200101818152505083806001019450505b8080600101915050610438565b5080935050505090565b60608060036040519080825280602002602001820160405280156104d55781602001602082028038833980820191505090505b50905060008090505b815181101561050d57808282815181106104f457fe5b60200260200101818152505080806001019150506104de565b508091505090565b6060600960008154811061052557fe5b906000526020600020018054600181600116156101000203166002900480601f0160208091040260200160405190810160405280929190818152602001828054600181600116156101000203166002900480156105c35780601f10610598576101008083540402835291602001916105c3565b820191906000526020600020905b8154815290600101906020018083116105a657829003601f168201915b5050505050905090565b6040518060400160405280600881526020017f4e657720596f726b000000000000000000000000000000000000000000000000815250600960008154811061061157fe5b90600052602060002001908051906020019061062e929190610972565b5060098060018154018082558091505090600182039060005260206000200160006040518060400160405280600581526020017f427573616e0000000000000000000000000000000000000000000000000000008152509091909150908051906020019061069d929190610972565b5050600a8060018154018082558091505090600182039060005260206000200160006040518060400160405280600881526020017f4e657720596f726b0000000000000000000000000000000000000000000000008152509091909150908051906020019061070d929190610972565b5050600a8060018154018082558091505090600182039060005260206000200160006040518060400160405280600781526020017f4265696a696e67000000000000000000000000000000000000000000000000008152509091909150908051906020019061077d929190610972565b5050565b6000600580549050905090565b6040518060a00160405280606460ff168152602001603c60ff168152602001605f60ff168152602001603260ff168152602001605060ff1681525060009060056107d99291906109f2565b50565b6060605a60056000815481106107ee57fe5b906000526020600020906003020160006003811061080857fe5b01819055506005805480602002602001604051908101604052809291908181526020016000905b82821015610892578382906000526020600020906003020160038060200260405190810160405280929190826003801561087e576020028201915b81548152602001906001019080831161086a575b50505050508152602001906001019061082f565b50505050905090565b6108a3610a37565b60405180604001604052806040518060400160405280600481526020017f393030300000000000000000000000000000000000000000000000000000000081525081526020016040518060400160405280600681526020017f5379646e6579000000000000000000000000000000000000000000000000000081525081525090506040518060400160405280600581526020017f53656f756c0000000000000000000000000000000000000000000000000000008152508160006002811061096757fe5b602002018190525050565b828054600181600116156101000203166002900490600052602060002090601f016020900481019282601f106109b357805160ff19168380011785556109e1565b828001600101855582156109e1579182015b828111156109e05782518255916020019190600101906109c5565b5b5090506109ee9190610a5e565b5090565b8260058101928215610a26579160200282015b82811115610a25578251829060ff16905591602001919060010190610a05565b5b509050610a339190610a83565b5090565b60405180604001604052806002905b6060815260200190600190039081610a465790505090565b610a8091905b80821115610a7c576000816000905550600101610a64565b5090565b90565b610aa591905b80821115610aa1576000816000905550600101610a89565b5090565b9056fea265627a7a72305820ed9a85d07e22dd2d7ed1e18cc89f1a25923ce89d7af2581e0577272589e01ac564736f6c63430005090032";
var _contract = web3.eth.contract(_abiArray);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
var _instance=_contract.new({data:"0x"+_bin,from:web3.eth.accounts[0],gas:1000000}, function(err, contract) {
    if (!err) {
        console.log("contractAddress: ", contract.address);
        console.log("transactionHash: ", contract.transactionHash);
    }
});

Overwriting src/ArrayTestDeploy.js


In [ ]:
!node src/ArrayTestDeploy.js

contractAddress:  undefined
transactionHash:  0x95b919d8fbd439ddcdbd6d0ddcedcb7a811d6ebce30cd03e05c09f618b41687b


### arraytest use

* ```getMathAbove70()``` 함수는 빈 배열을 출력하고 있어 문제가 있다.

In [ ]:
%%writefile src/ArrayTestUse.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8545"));
var arraytestContract = web3.eth.contract([{"constant":true,"inputs":[],"name":"getMarksArr","outputs":[{"name":"","type":"uint256[3][]"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"getMathAbove70","outputs":[{"name":"","type":"int256[]"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":true,"inputs":[],"name":"getArrMemory","outputs":[{"name":"","type":"uint256[]"}],"payable":false,"stateMutability":"pure","type":"function"},{"constant":true,"inputs":[],"name":"getCities","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"setCities","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getMarksLength","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"setMathMarks","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"updateMarks","outputs":[{"name":"","type":"uint256[3][]"}],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getDynamicArrMemory","outputs":[],"payable":false,"stateMutability":"pure","type":"function"}]);
var arraytest = arraytestContract.at("0xc5a134071d8d9d64f582d88ece87d384ab1fe68b");
console.log(arraytest.setMathMarks({from:web3.eth.accounts[0]}));
console.log(arraytest.getMathAbove70());
console.log(arraytest.getArrMemory().toString());
console.log(arraytest.getMarksArr().toString());

In [ ]:
```
0x9e05bb4d68e98b728f2acfb9eeb22b1101687b92e1dfa410ddbc0cbeb27a4fcb
[]
0,1,2
100,80,95,20,30,40
```

> arraytest.getMarksArr()                      출력하기 위해서는 toString()
[ [ BigNumber { s: 1, e: 2, c: [Array] },
    BigNumber { s: 1, e: 1, c: [Array] },
    BigNumber { s: 1, e: 1, c: [Array] } ],
  [ BigNumber { s: 1, e: 1, c: [Array] },
    BigNumber { s: 1, e: 1, c: [Array] },
    BigNumber { s: 1, e: 1, c: [Array] } ] ]
> arraytest.getMarksLength()
BigNumber { s: 1, e: 0, c: [ 2 ] }
> arraytest.getMarksLength().toString()
'2'
> arraytest.getMarksArr().toString()
'100,80,95,20,30,40'
> arraytest.getArrMemory();
[ BigNumber { s: 1, e: 0, c: [ 0 ] },
  BigNumber { s: 1, e: 0, c: [ 1 ] },
  BigNumber { s: 1, e: 0, c: [ 2 ] } ]
> arraytest.getArrMemory().toString();
'0,1,2'
> arraytest.getArrMemory().pop().toString()
'2'

배열을 검색하는 것은 비용이 많이 발생한다. 
mapping을 사용하거나, 클라이언트 측에서 배열 데이터를 검색하는 편으로 하자.

문자열 비교 '=='는 사용할 수 없다. ```keccak256()``` 해싱을 한 후 비교해야 한다.

In [2]:
%%writefile src/Members.sol
pragma solidity ^0.5.0;
contract Members{
    address owner;
    event printAddr(address arg);
    struct Member{
        uint id;
        string name;
    }
    Member[] public memberArr;
    constructor() public {
        owner=msg.sender;
    }
    function del() public {
        delete memberArr;
    }
    function delOne(uint i) public{
        delete memberArr[i];
    }
    function add(uint id,string memory name) public {
        memberArr.push(Member(id,name));
    }
    //return Member
    function getMember(string memory who) view public returns(uint, string memory) {
        uint _id;
        string memory _name;
        for(uint i=0;i<memberArr.length;i++) {
            _name=memberArr[i].name;
            if(keccak256(abi.encodePacked(_name))==keccak256(abi.encodePacked(who))) {
                _id=memberArr[i].id;
                _name=memberArr[i].name;
            }
        }
        return (_id,_name);
    }
    function compareStr(string memory s1, string memory s2) pure public returns(bool) {
        return keccak256(abi.encodePacked(s1))==keccak256(abi.encodePacked(s2));
    }
    function compareBytes(bytes memory b1, bytes memory b2) pure public returns(bool) {
        return keccak256(b1) == keccak256(b2);
    }
    function getLength() view public returns(uint) {
        return memberArr.length;
    }
}

Writing src/Members.sol


### 9.7.2 mapping
<a id='S.5.2'></a>

```mapping```은 `키`와 키에 해당하는 `값`을 쌍으로 저장한다. ```=>``` 기호를 사용하여, 좌측에 키를 오른쪽에 값을 적어준다.
아래는 (이름, 잔고)를 저장하는 mapping이다.

```python
mapping (string => uint) public balances;
balances["jsl"] = 100;
```

## 실습: 은행

앞서 은행에서는 한 계정에서의 입금, 출금을 실행해 보았다.
여러 계정이 있는 경우, mapping을 이용해 잔고를 관리할 수 있다.

```mapping(address=>uint) balanceOf;``` 라고 선언하고
```balanceOf[<address>]=amount;```로 잔고를 저장할 수 있다.

증액, 감액을 하려면:
```python
uint balanceToAdd=111;
balanceOf[<address>] += balanceToAdd;
```

* adddress 자신의 주소를 따옴표 없이, 금액도 따옴표 없이 정수

In [1]:
%%writefile src/BankV4.sol
//check for Coin.sol (ch12) and BankV3.sol 201906
pragma solidity ^0.5.0;
contract BankV3 {
    address minter;
    mapping (address => uint) balanceOf;
    constructor() public {
        minter = msg.sender;
    }
    function mint(address owner, uint amount) public {
        require(msg.sender == minter);
        balanceOf[owner] += amount;
    }
    function send(address receiver, uint amount) public {
        require(balanceOf[msg.sender]>=_value);
        balanceOf[msg.sender] -= amount; // Subtract from the sender
        balanceOf[receiver] += amount;   // Add the same to the recipient
    }
    function queryBalance(address addr) public view returns (uint) {
        return balanceOf[addr];
    }
}


Writing src/BankV4.sol


* 중복키 확인

맵은 그 특성상 중복키는 허용되지 않는다. 동일한 키로 당연히 데이터를 입력할 수가 없다.
맵에 이미 키가 존재하는지 확인할 수 있는 ```containsKey()```와 같은 지원 함수가 없다.
필드 ```isMember```를 설정해놓고 그 값을 확인하는 방법으로 하자.
```
mapping(address=>Member) memberMap;
struct Member {
	uint id;
	string name;
	bool isMember;
}
if (memberMap(<address>).isMember) 중복키 존재함
```

* 삭제

키삭제는 쉽지 않다. 다른 언어에서는 ```remove()```함수가 지원되어, 간편하게 키를 지울 수 있는 것과 비교된다.

```
delete pendingAccessRequest[key];
```

#### bidirectional map

주소에 대해 회원(아이디, 이름)을 저장하는 ```map```이다.
이 경우 '이름'으로 '주소'를 검색하는 기능은 ```bidiretional map```으로 구현하였다.

줄 | 설명
-----|-----
9 | 주소의  ```Member strcut ```을 조회하는 map
10 | 이름의 주소를 조회하는 map
11~16 ```addMember() ``` |  ```Member struct```을 추가하고 ```getMemberByName()``` 함수에서 이름을 입력하면 주소를 조회할 수 있도록 저장
17~19 ```getMemberByName()``` | bidirectional map에서 이름에 해당하는 주소를 조회
20~23 ```getMember()``` | 주소에 해당하는 ```Member struct```을 조회

In [1]:
%%writefile src/MembersMap.sol
pragma solidity ^0.5.0;

contract MembersMap {
    struct Member {
        uint id;
        string name;
    }
    //bidrectional map
    mapping(address=>Member) memberMap;
    mapping(string=>address) addressByName;
    function addMember (uint _id, string memory _name) public {
       Member memory m=Member(_id, _name);
       memberMap[msg.sender]=m;
       //saving into a bidiretional map to get address by name
       addressByName[_name]=msg.sender;
    }
    function getMemberByName(string memory _name) public view returns(address) {
        return addressByName[_name];
    }
    function getMember(address addr) view public returns (uint, string memory) {
        Member memory m=memberMap[addr];
        return (m.id, m.name);
    }
}

Writing src/MembersMap.sol


## 9.8 Exception
<a id='S.6'></a>

오류는 발생하기 마련이다. 컴파일 오류는 수정하기 비교적 용이하다. 그러나 실행시점에 발생하는 오류는 잡아내기 어렵다.
다른 언어에서 보통 제공하는 ```try..catch``` 구문도 제공되지 않는다.
버전 4.10 이전에는 ```throw``` 명령문이 사용되었지만 문제가 있었다.
지급한 gas를 다 소비하고 오류발생 이전의 상태로 돌려놓았다.

```python
if (msg.sender != owner) { throw; }
```

버전 4.10이후에는 ```throw```구문은 더 이상 사용되지 않고 다음 구문이 사용된다.


예외는 배열이 범위를 넘어서는 경우와 같이 실행되면서 발생할 수 있다.
반면에 ```out-of-gas errors```, ```divide by zero```와 같이 어떤 특정 조건에서 예외를 발생시킬 수 있다.
이 경우 실행이 중지되고, 원상복구 (상태변수와 잔고)된다.

* ```assert()``` 미사용 gas는 반환되지 않고, 모두 소모된다. 상태는 원래대로 복원된다.
* ```require()``` | ```true```, ```false```로 반환된다. ```false```인 경우 예외가 발생하고 실행이 중지된다. 미사용 gas를 호출자에게 반환하고 상태를 원래대로 복원한다.
* ```revert()```  ```require()``` 함수와 동일한 의미이지만, 어떤 평가를 하지 않는다는 점이 다르다. 즉, ```revert`` 명령문은 미사용 gas를 호출자에게 반환하고 상태를 원래대로 복원한다.

명령문 | gas 반환 | 원복 | 사용하는 경우
-----|-----|-----|-----
```assert()``` | NO | YES | overflow/underflow, invariants 등 어떤 오류가 발생하는 것을 예방하는 경우
```revert()``` | YES | YES | ```require()```는 참, 거짓을 평가하지만 조건문 없이 수행되기 때문에 이진비교 이상을 하는 경우
```require()``` | YES | YES | 사용자 입력 , 상태변수 값, 반환 값 등 조건을 검증하는 경우

In [3]:
%%writefile src/ExceptionTest.sol
pragma solidity *0.5.0;

contract ExceptionTest {
    address owner;
    constructor() public {
        owner=msg.sender;
    }
    function requireException() view public returns(string memory) {
        //if (msg.sender != owner) { throw; }
        require(msg.sender != owner, "Sorry! You are NOT owner. Require failed...");
        return "asserted";
    }
    function assertException() view public returns(string memory) {
        assert(msg.sender == owner);
        return "asserted";
    }
    function revertException() view public returns(string memory) {
        if(msg.sender != owner) {
            revert("Sorry! You are NOT owner. Reverted...");
            return "reverted";
        } else {
            return "not reverted";
        }
    }
    function raiseException() view public {
        int x=0;
        int y=0;
        x/y;    //divide by zero
    }
}

Overwriting src/ExceptionTest.sol
